In [71]:
import sys
sys.path.append("/home/ubuntu/roaddetection/")

In [72]:
#%load_ext autoreload
#%autoreload 2

from src.models.data import *
from src.models.model import *
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

%matplotlib inline

In [88]:
import os, shutil

original_dataset_dir = "../../data/train"

dirs = []

base_dir = "../../data"

train_dir = os.path.join(base_dir, "train")
dirs.append(train_dir)
validation_dir = os.path.join(base_dir, "validate")
dirs.append(validation_dir)
test_dir = os.path.join(base_dir, "test")
dirs.append(test_dir)

for directory in dirs:
    if not os.path.exists(directory):
       os.mkdir(directory)
    
fnames = ["20180419_074323_0c43_3B_{0:04d}.tif".format(i) for i in range(70,109)]
for fname in fnames:
    for file_type in ["sat", "map", "sat_rgb"]:
        src = os.path.join(train_dir, file_type, fname)
        dest = os.path.join(validation_dir, file_type, fname)
        shutil.move(src, dest)

In [91]:
for directory in dirs:
    for file_type in ["sat", "map", "sat_rgb"]:
        target = os.path.join(directory, file_type)
        print(target, ":", len(os.listdir(target)))
    
print("Done.")

../../data/train/sat : 71
../../data/train/map : 72
../../data/train/sat_rgb : 71
../../data/validate/sat : 39
../../data/validate/map : 39
../../data/validate/sat_rgb : 39
../../data/test/sat : 110
../../data/test/map : 110
../../data/test/sat_rgb : 110
Done.


In [73]:
data_gen_args = dict(
                    data_format="channels_last"
 )
train_generator = trainGenerator(5,'../../data/train','sat','map',
                        data_gen_args,save_to_dir = None,image_color_mode="rgba", target_size=(512,512))

In [92]:
validation_generator= trainGenerator(5,'../../data/validate','sat','map',
                        data_gen_args,save_to_dir = None,image_color_mode="rgba", target_size=(512,512))

In [74]:
def get_fcn_vgg16_32s(input_size, n_classes):
    inputs = Input(input_size)

    x = BatchNormalization()(inputs)
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#     # Block 4
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#     # Block 5
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    x = Conv2D(512, (2, 2), activation='relu', padding="same")(x)
    
    x = Conv2DTranspose(n_classes, kernel_size=(64, 64), strides=(8, 8), activation='linear', padding='same')(x)
   
    model = Model(input=inputs, output=x)
        
    model.compile(optimizer=SGD(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [75]:
model = get_fcn_vgg16_32s((512, 512, 4),1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 512, 512, 4)       0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 512, 512, 4)       16        
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      2368      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 128, 128, 256)     295168    
__________

/Users/kprakash/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [76]:
model_checkpoint = ModelCheckpoint('../../models/fcnn_29_08_22_00.hdf5', monitor='loss',verbose=1, save_best_only=True)
history = model.fit_generator(
    myGene,
    steps_per_epoch=100,
    epochs=5,
    callbacks=[model_checkpoint]
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/5
Found 110 images belonging to 1 classes.
Found 110 images belonging to 1 classes.
100/100 [==============================] - 767s 8s/step - loss: 0.1367 - acc: 0.9720

Epoch 00001: loss improved from inf to 0.13671, saving model to ../../models/fcnn_29_08_22_00.hdf5
Epoch 2/5
 44/100 [============>.................] - ETA: 6:57 - loss: 0.1251 - acc: 0.9726

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    plt.plot(history["acc"], label="acc")
    plt.plot(history["val_acc"], label="val_acc")
    plt.legend()
    plt.show()
    plt.close()
    
    plt.plot(history["loss"], label="loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend()
    plt.show()
    plt.close()

plot_history(history.history)

In [ ]:
testGene = testGenerator("../../data/test/sat", num_image=50,target_size=(512,512),as_gray=False)
results = model.predict_generator(testGene, steps=50)
saveResult("../../data/test/predict",results)